In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import random
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

Goals

    -implement spectral clustering
    -implement belief propagation
    -implement it with power method

Vanilla Belief propagation algorithm

Input: a stochastic block model (adjaceny matrix)

1) for each v, v' adajecnt, randomly draw y^1_vv' from N(0,1)

2) find all cycles in G of length r or less.  To do this, we may have to multiply r times, but avoid the backtracking ones.  

3) for each 1<t<m and each adjacent v, v' set y^t_vv' = sum of all the y's up to that t....since we seeded it with N(0,1) we will have the value for y^1.  Howver, if (v,v') is part of a cycle of r or less, don't do this.  Instead, 
4) if your edge is part of a r cycle, then we subtract from the previous sum, the accumulated influence of the other adjacent edge in our cycle, unless the r cycle is the same lenght as the time steps, in which case we just subtract the original randomized N(0,1)

5) Let Y be the matrix composed of all y_vv' summed up with edge vertices




In [2]:
#first try implementing power method in tensorflow

#feed in graphs, as in adjaceny matrices, or just nodes and connected edges?

#each layer will feed in a random vector, then multiply our matrix by the random vector
#many times.  At the end, we subtract away the orignal vector component

#next layer we just randomize another vector...

#how many layers.  The depth can be learned by adding identify layers, which is essentially 
#self multiplication.  

#adjaceny matrix to graph laplacian 

#suppose for simplicity that our graph has the same dimnension

#10by10 graph, so feed adjacency matrix
dim_graph = 10
dim_subspace = 3

x = tf.placeholder("float", shape = [dim_graph, dim_graph])
y = tf.placeholder("float", shape = [None, dim_graph])
#y is our community classification answer



In [3]:
#initialize the k random vectors (whcih should be tuned)
#that we are feeding to the laplacian operator
d = {}
for i in range(1,dim_subspace):
    d["v{}".format(i)] = tf.Variable(tf.random_normal(shape=[1, dim_graph], 
                         mean=0.0, 
                         stddev = 1.0))

#may have to define another way....

In [64]:
#laplacian

#diffusion:

def DiffusionVertex(vertex, graph, vector):
    #assume graph is in form of adjacency matrix 
    neighbour_vector = tf.slice(graph, begin=[vertex,0], size=[1,dim_graph])
    prod = tf.mul(vector, neighbour_vector)
    diffusion = tf.reduce_sum(prod, 1)
    return(diffusion)

def GraphLaplacian(graph, vector):
    #takes in graph and vector on graph indices, returns new vector
    tmp_v = tf.constant(range(dim_graph))
    def DiffusionVertex_1(vertex):
        return DiffusionVertex(vertex, graph, vector)
    diffusion_vector = tf.map_fn(DiffusionVertex_1, tmp_v) 
    
    return tf.reduce_sum(diffusion_vector)

def euclidean_norm(tensor): #need to have this for tf to work
    square_tensor = tf.square(tensor)
    norm_squared = tf.reduce_sum(square_tensor)
    euclidean_norm = tf.sqrt(norm_squared)
    return euclidean_norm


    

In [65]:
elems = np.array([1, 2, 3, 4, 5, 6], dtype="float32")
squares = tf.map_fn(lambda x: x * x, elems)

sess = tf.Session()
sess.run(squares)


array([  1.,   4.,   9.,  16.,  25.,  36.], dtype=float32)

In [66]:
#test to see if your diffusion operator works

x = tf.constant([[1,2,3], [2,3,4], [3,4,5]], dtype=tf.float32)
y = tf.Variable(tf.random_normal(shape=[3], 
                         mean=0.0, 
                         stddev = 1.0))

init_op = tf.initialize_all_variables()
answer = tf.mul(x,tf.slice(x, [2,0], [1,3])),tf.slice(x, [2,0], [1,3])
with tf.Session() as sess:
    sess.run(init_op)
    print sess.run(answer)
    

[array([[  3.,   8.,  15.],
       [  6.,  12.,  20.],
       [  9.,  16.,  25.]], dtype=float32), array([[ 3.,  4.,  5.]], dtype=float32)]


In [67]:
dim_graph = 3
dim_subspace = 3

x = tf.constant([[1,2,3], [2,3,4], [3,4,5]])
x = tf.cast(x, tf.float32)
f_x = tf.Variable(tf.random_normal(shape=[3], 
                         mean=0.0, 
                         stddev = 1.0))

blah = [DiffusionVertex(i, graph=x, vector=f_x) for i in range(dim_graph)]
blah = tf.reduce_sum(tf.pack(blah))

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    print sess.run(blah)


-4.58576


In [68]:
#now let continue with the powermethod from the above:
#since the largest k eigenvalues is supposed to come from the adjacney matrix, 
#we leave the diffusion component for now

dim_graph = 3
dim_subspace = 3

#x = tf.placeholder(tf.float32, shape=(dim_graph, dim_graph))
x = tf.cast(x, tf.float32)

answer_lst = []

lmbda_lst = []

f = {}
for i in range(1,dim_subspace+1):
    f["v{}".format(i)] = tf.Variable(tf.random_normal(shape=[dim_graph, 1], 
                         mean=0.0, 
                         stddev = 1.0))
    
    

adjacency_vector = tf.matmul(x, f["v1"])
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)
print adjacency_vector
answer_lst.append(adjacency_vector)

#lmbda_lst.append(euclidean_norm(tf.matmul(x, adjacency_vector)) #since it is already normalized

                 
adjacency_vector = tf.sub(f["v2"], adjacency_vector)
adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

answer_lst.append(adjacency_vector)
#lmbda_lst.append(euclidean_norm(tf.matmul(x, adjacency_vector)) #since it is already normalized


adjacency_vector = tf.sub(tf.sub(f["v3"], adjacency_vector), answer_lst[1])
adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

adjacency_vector = tf.matmul(x, adjacency_vector)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

answer_lst.append(adjacency_vector)
#lmbda_lst.append(euclidean_norm(tf.matmul(x, adjacency_vector)) #since it is already normalized

                 

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    print sess.run(answer_lst, feed_dict = {x: [[1,2,3], [2,3,4], [3,4,5]]})


Tensor("div_41:0", shape=(3, 1), dtype=float32)
[array([[ 0.38511941],
       [ 0.5595153 ],
       [ 0.73391116]], dtype=float32), array([[-0.38507888],
       [-0.55950832],
       [-0.73393774]], dtype=float32), array([[-0.38455883],
       [-0.55941874],
       [-0.73427862]], dtype=float32)]


In [69]:
#

In [70]:
np.matrix([[1,2,3], [2,3,4], [3,4,5]])*np.matrix([[ 2.25037885], [2.82363868], [3.39689851]])


matrix([[ 18.08835174],
        [ 26.55926778],
        [ 35.03018382]])

In [71]:

y = tf.matmul(tf.constant([[1.0,2.0,3.], [2.,3.,4.], [3.,4.,5.]], shape=[3,3]),
          tf.constant([ 2.25037885, 2.82363868, 3.39689851], shape=[3,1]))
with tf.Session() as sess:
    print sess.run(y)
    
    #ok, they match, so at least that is not wrong....
    
    

[[ 18.0883522 ]
 [ 26.55926895]
 [ 35.0301857 ]]


In [138]:
x = np.random.rand(6,6)
eigenValues,eigenVectors = np.linalg.eig(x)
y = eigenVectors[:, eigenValues.argmax()]

print x
print y

[[ 0.67316258  0.77078715  0.84736085  0.14172805  0.08583192  0.80161512]
 [ 0.80306613  0.86274484  0.22028874  0.01986374  0.04525851  0.68746425]
 [ 0.63863501  0.21335319  0.95988464  0.52516139  0.29216087  0.15448989]
 [ 0.29623571  0.97427035  0.69251197  0.82959694  0.96867847  0.11098715]
 [ 0.47638091  0.97155361  0.53450427  0.846212    0.8239428   0.72508975]
 [ 0.63984247  0.29791923  0.17945452  0.2135168   0.97254578  0.59230027]]
[-0.35918558+0.j -0.27768321+0.j -0.33576492+0.j -0.49008081+0.j
 -0.54394370+0.j -0.38090121+0.j]


In [139]:
#now let try to get the first egenvector right with some backprop




dim_graph = len(x)
dim_subspace = 1
    
x = tf.cast(x, tf.float32)
y = tf.cast(y, tf.float32)

f = {}
for i in range(1,dim_subspace+1):
    f["v{}".format(i)] = tf.Variable(tf.random_uniform(shape = [dim_graph,1],
                                                       minval=-1,
                                                       maxval = 1,
                                                       dtype = tf.float32))
y = f["v1"]
   

adjacency_vector = tf.matmul(x, y)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

cost = tf.reduce_mean(tf.square(y-adjacency_vector))                
optimizer = tf.train.GradientDescentOptimizer(0.01)

train = optimizer.minimize(loss = cost, var_list = [f["v1"]])

init_op = tf.initialize_all_variables()


sess = tf.Session()
sess.run(init_op)

for i in xrange(1000):
    sess.run(train)
    if i%500==0:
        print sess.run([adjacency_vector, f["v1"]])
#seems to be accurate, though not really fast.

[array([[ 0.39295191],
       [ 0.28276429],
       [ 0.41409135],
       [ 0.08971775],
       [ 0.36310893],
       [ 0.6739912 ]], dtype=float32), array([[ 0.73673558],
       [-0.76855552],
       [ 0.18964009],
       [-0.01761648],
       [ 0.50768691],
       [ 0.74849772]], dtype=float32)]
[array([[ 0.38408989],
       [ 0.28253925],
       [ 0.3526527 ],
       [ 0.42363718],
       [ 0.52225506],
       [ 0.44279084]], dtype=float32), array([[ 0.39211223],
       [-0.01557965],
       [ 0.33084738],
       [ 0.28736705],
       [ 0.53432196],
       [ 0.52184629]], dtype=float32)]


In [117]:
#now let try to get an eigenspace with backprop
dim_graph = 3
dim_subspace = 2


x = [[1,0,0], [0,1,1], [0,0,5]]
eigenValues,eigenVectors = np.linalg.eig(x)
y = eigenVectors[:, eigenValues.argmax()]
    
x = tf.cast(x, tf.float32)
y = tf.cast(y, tf.float32)

f = {}
for i in range(1,dim_subspace+1):
    f["v{}".format(i)] = tf.Variable(tf.random_uniform(shape = [dim_graph,1],
                                                       minval=-1,
                                                       maxval = 1,
                                                       dtype = tf.float32))
y = f["v1"]
   

adjacency_vector = tf.matmul(x, y)
adjacency_vector = adjacency_vector/euclidean_norm(adjacency_vector)

cost = tf.reduce_mean(tf.square(y-adjacency_vector))                
optimizer = tf.train.GradientDescentOptimizer(0.01)

train = optimizer.minimize(loss = cost, var_list = [f["v1"]])

init_op = tf.initialize_all_variables()


sess = tf.Session()
sess.run(init_op)

for i in xrange(10000):
    sess.run(train)
    if i%500==0:
        print sess.run([adjacency_vector, f["v1"]])
#seems to be accurate, though not really fast.

[ 0.          0.24253563  0.9701425 ]


The above does not do well with certain vectors, but with the identity it is not totally off.....

Problem with using tf.map_fn below.  Using list instead, however this the step that is parallelizable, hence not sure that writing in terms of for loop is ideal... How will tensorflow handle it?

In [129]:
x = np.random.rand(2,2)
len(x)

2

In [10]:
dim_graph = 3
dim_subspace = 3

x = tf.constant([[1,2,3], [2,3,4], [3,4,5]])
x = tf.cast(x, tf.float32)
y = tf.Variable(tf.random_normal(shape=[dim_graph], 
                         mean=0.0, 
                         stddev = 1.0))

tmp_v = np.array(range(dim_graph), dtype="int")

def DiffusionVertex_1(vertex):
    return DiffusionVertex(vertex, x, y)

diffusion_vector = [DiffusionVertex_1(i) for i in range(dim_graph)]
#diffusion_vector = tf.scan(DiffusionVertex_1, tmp_v)
diffusion_vector = tf.pack(diffusion_vector)

#answer = tf.shape(tmp_v)
answer = diffusion_vector

#answer = tf.shape(diffusion_vector)
answer = tf.reduce_sum(diffusion_vector)

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    print sess.run(answer)

-5.3163
